In [1]:
import pandas as pd
import requests
from itertools import combinations

In [2]:
api_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

response = requests.get(api_url)

if response.status_code == 200:
    data = response.json()
    print("Succesfully fetched data!")
else:
    print(f"Error while fetching data. Status code: {response.status_code}")

Succesfully fetched data!


In [3]:
print(data.keys())

dict_keys(['chips', 'events', 'game_settings', 'game_config', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])


In [4]:
players = pd.DataFrame(data['elements'])
players.head()

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,True,False,0.0,0.0,438098,0,0,-1,1,0,...,137,82,674,304,682,310,636,278,0.00,0.00
1,True,True,100.0,100.0,205651,0,0,-2,2,0,...,32,15,303,38,459,49,214,37,0.74,0.37
2,True,True,75.0,75.0,226597,0,0,2,-2,2,...,66,3,76,17,31,4,13,5,1.08,0.33
3,True,True,100.0,100.0,219847,0,0,-1,1,1,...,13,5,88,18,37,14,37,10,1.01,0.29
4,True,False,0.0,0.0,463748,0,0,0,0,0,...,638,63,425,51,514,58,558,68,0.00,0.00


In [5]:
print("Players DataFrame Columns:")
print(players.columns.tolist())


Players DataFrame Columns:
['can_transact', 'can_select', 'chance_of_playing_next_round', 'chance_of_playing_this_round', 'code', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo', 'points_per_game', 'removed', 'second_name', 'selected_by_percent', 'special', 'squad_number', 'status', 'team', 'team_code', 'total_points', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'region', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists', 'expected_goal_involvements', 'expected_goals_conceded', 'influence_

In [6]:
position_map = {
    1: 'GK',
    2: 'Defender',
    3: 'Midfielder',
    4: 'Forward'
}

players_selected = players[
    ['first_name', 'second_name', 'team_code', 'element_type', 'total_points', 'points_per_game', 'now_cost', 'chance_of_playing_next_round', 'chance_of_playing_this_round']
].copy()

players_selected['now_cost'] = players_selected['now_cost'] / 10

players_selected['element_type'] = players_selected['element_type'].map(position_map)

players_selected[['total_points', 'points_per_game', 'now_cost']] = players_selected[
    ['total_points', 'points_per_game', 'now_cost']
].apply(pd.to_numeric, errors='coerce')

players_selected = players_selected.loc[players_selected['points_per_game'] > 0.0]

players_selected['full_name'] = players_selected['first_name'] + ' ' + players_selected['second_name']

print(players_selected)


         first_name             second_name  team_code element_type  \
1           Gabriel       Fernando de Jesus          3      Forward   
2           Gabriel    dos Santos Magalhães          3     Defender   
3               Kai                 Havertz          3      Forward   
5           Jurriën                  Timber          3     Defender   
6        Jorge Luiz            Frello Filho          3   Midfielder   
..              ...                     ...        ...          ...   
686          Jørgen           Strand Larsen         39      Forward   
687    Toti António                   Gomes         39     Defender   
689           André  Trindade da Costa Neto         39   Midfielder   
690  Carlos Roberto            Forbs Borges         39   Midfielder   
691           Alfie                    Pond         39     Defender   

     total_points  points_per_game  now_cost  chance_of_playing_next_round  \
1               9              0.8       6.8                         

In [7]:
max_indeces = players_selected[['total_points', 'points_per_game', 'now_cost']].idxmax()
max_rows = players_selected.loc[max_indeces]
print(max_rows)


    first_name second_name  team_code element_type  total_points  \
401    Mohamed       Salah         14   Midfielder           151   
401    Mohamed       Salah         14   Midfielder           151   
428     Erling     Haaland         43      Forward            98   

     points_per_game  now_cost  chance_of_playing_next_round  \
401             10.8      13.3                           NaN   
401             10.8      13.3                           NaN   
428              6.5      14.9                         100.0   

     chance_of_playing_this_round       full_name  
401                           NaN   Mohamed Salah  
401                           NaN   Mohamed Salah  
428                         100.0  Erling Haaland  


In [8]:
goalkeepers = players_selected[players_selected['element_type'] == 'GK']

goalkeepers.head()

,first_name,second_name,team_code,element_type,total_points,points_per_game,now_cost,chance_of_playing_next_round,chance_of_playing_this_round,full_name
11,David,Raya Martin,3,GK,58,3.9,5.6,100.0,100.0,David Raya Martin
21,Norberto,Murara Neto,3,GK,7,3.5,4.2,100.0,100.0,Norberto Murara Neto
52,Emiliano,Martínez Romero,7,GK,43,2.9,5.0,100.0,100.0,Emiliano Martínez Romero
56,Robin,Olsen,7,GK,1,1.0,4.5,100.0,100.0,Robin Olsen
94,Mark,Travers,91,GK,10,3.3,4.4,NaN,NaN,Mark Travers


In [9]:
print(f"Total GK's: {len(goalkeepers)}")

Total GK's: 34


In [10]:
min_indeces = goalkeepers[['total_points', 'points_per_game', 'now_cost']].idxmin()
min_rows = goalkeepers.loc[min_indeces]
print(min_rows)


    first_name second_name  team_code element_type  total_points  \
56       Robin       Olsen          7           GK             1   
56       Robin       Olsen          7           GK             1   
413  Vítezslav       Jaros         14           GK             1   

     points_per_game  now_cost  chance_of_playing_next_round  \
56               1.0       4.5                         100.0   
56               1.0       4.5                         100.0   
413              1.0       4.0                           NaN   

     chance_of_playing_this_round        full_name  
56                          100.0      Robin Olsen  
56                          100.0      Robin Olsen  
413                           NaN  Vítezslav Jaros  


In [14]:
# Generate all combinations of 2 goalkeepers
gk_combinations = list(combinations(goalkeepers.itertuples(), 2))

optimal_goalkeepers = None
best_total_points = 0
best_comb_cost = 0
best_cpp = 100
max_cost = 15

for gk_1, gk_2 in gk_combinations:
    total_cost = gk_1.now_cost + gk_2.now_cost
    if total_cost > max_cost:
        continue

    total_points = gk_1.total_points + gk_2.total_points
    starter, bench = sorted((gk_1, gk_2), key=lambda gk: gk.points_per_game, reverse=True)
    cpp = total_cost / total_points

    if cpp < best_cpp:
        best_cpp = cpp
        best_total_points = total_points
        best_comb_cost = total_cost
        optimal_goalkeepers = (starter, bench)

# Unpack and print the results
starter, bench = optimal_goalkeepers
print(f"Total points: {best_total_points}")
print(f"Total cost: {best_comb_cost} \n")
print(f"Cost/point: {best_cpp} \n")

print(f"STARTER: {starter.first_name} {starter.second_name} - {starter.element_type} - Team Code: {starter.team_code} - Total points: {starter.total_points} - Cost: £{starter.now_cost}m")
print(f"BENCH: {bench.first_name} {bench.second_name} - {bench.element_type} - Team Code: {bench.team_code} - Total points: {bench.total_points} - Cost: £{bench.now_cost}m")


Total points: 127
Total cost: 9.9 

Cost/point: 0.07795275590551182 

STARTER: André Onana - GK - Team Code: 1 - Total points: 65 - Cost: £5.2m
BENCH: Matz Sels - GK - Team Code: 17 - Total points: 62 - Cost: £4.7m
